<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

gen_output = 0

Run date time (IST): 2023-07-19 15:01:39


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [4]:
# set start and end date
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')



In [5]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv')

myts = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myTransactionsReport.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'BAJAJFINSV',
       'BAJAJHIND', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP', 'CAMS',
       'CDSL', 'CLEAN', 'DMART', 'EICHERMOT', 'EMAMILTD', 'FINEORG',
       'GICRE', 'GILLETTE', 'GLAND', 'HDFCAMC', 'HDFCBANK', 'ICICIGI',
       'INDIGOPNTS', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITBEES',
       'JMFINANCIL', 'JUBLFOOD', 'LTIM', 'LUXIND', 'METROPOLIS', 'NIACL',
       'NMDC', 'PAGEIND', 'PIDILITIND', 'PNB', 'RAJESHEXPO', 'RELAXO',
       'ROSSARI', 'SAIL', 'STARHEALTH', 'SUNTV', 'TCS', 'TEAMLEASE',
       'UJJIVANSFB', 'VAIBHAVGBL', 'VALIANTORG', 'WIPRO'], dtype=object)

In [6]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Stock'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    tmp['Target'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Target'].values[0]
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [7]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [8]:
# portfolio stocks

df_prec_dev["Target"] = df_prec_dev["Target"].fillna(df_prec_dev["Max"])

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Target'] * tmp_df['Shares']
tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%']

investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
previous = round(sum(tmp_df['Prev_Close']*tmp_df['Shares']),0)

today_pnl_amount = current-previous
today_pnl_percentage = round((current-previous)*100/previous,2)

curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['Weightage%'],0)

print('Investment: ', investment)
print('Current: ', current)
print('Today PnL: ', today_pnl_amount)
print('Today PnL %: ', today_pnl_percentage)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)

qualified stocks: 49
Investment:  3300385.0
Current:  3482978.0
Today PnL:  13824.0
Today PnL %:  0.4
Current PnL:  182593.0
Current PnL %:  5.53
Estimate:  5934255.0
Estimate PnL:  2633870.0
Estimate PnL %:  79.8


In [9]:
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Target,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
17,5730.00,GILLETTE.NS,7529.0,5873.25,4475.85,21.0,V40,ATH,7529.0,93992.85,120330.00,123338.25,158109.0,-2.44,28.02,68.21,40.19,2.85,80.0
19,2414.80,HDFCAMC.NS,3518.0,2383.70,1973.85,50.0,V40,ATH,3518.0,98692.50,120740.00,119185.00,175900.0,1.30,22.34,78.23,55.89,2.99,67.0
36,64.00,PNB.NS,95.0,62.35,54.97,1400.0,VR,ATH,95.0,76958.00,89600.00,87290.00,133000.0,2.65,16.43,72.82,56.39,2.33,38.0
21,1366.30,ICICIGI.NS,1613.0,1365.20,1177.58,84.0,V40,ATH,1613.0,98916.72,114769.20,114676.80,135492.0,0.08,16.03,36.98,20.95,3.00,48.0
45,42.40,UJJIVANSFB.NS,57.0,42.25,36.90,1850.0,V40N,ATH,57.0,68265.00,78440.00,78162.50,105450.0,0.36,14.91,54.47,39.56,2.07,31.0
0,392.90,5PAISA.NS,553.0,392.05,347.06,228.0,V40N,ATH,553.0,79129.68,89581.20,89387.40,126084.0,0.22,13.21,59.34,46.13,2.40,32.0
4,1643.75,BAJAJFINSV.NS,1905.0,1610.25,1454.93,52.0,V40,ATH,1905.0,75656.36,85475.00,83733.00,99060.0,2.08,12.98,30.93,17.95,2.29,30.0
10,1210.45,CDSL.NS,1648.0,1229.75,1082.25,70.0,VR,ATH,1648.0,75757.50,84731.50,86082.50,115360.0,-1.57,11.85,52.28,40.43,2.30,27.0
24,1473.10,INFY.NS,1848.0,1422.95,1319.64,81.0,V40,ATH,1848.0,106890.84,119321.10,115258.95,149688.0,3.52,11.63,40.04,28.41,3.24,38.0
7,1700.05,BATAINDIA.NS,2173.0,1692.95,1523.62,100.0,V40,ATH,2173.0,152362.00,170005.00,169295.00,217300.0,0.42,11.58,42.62,31.04,4.62,53.0


In [10]:
tmp_df.sort_values(by = 'Juice Left%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Target,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
35,2683.90,PIDILITIND.NS,2902.0,2639.60,2504.06,30.0,V40,ATH,2902.0,75121.80,80517.00,79188.00,87060.0,1.68,7.18,15.89,8.71,2.28,16.0
43,3470.25,TCS.NS,3857.0,3491.70,3231.00,16.0,V40,ATH,3857.0,51696.00,55524.00,55867.20,61712.0,-0.61,7.40,19.37,11.97,1.57,12.0
13,3330.00,EICHERMOT.NS,3850.0,3349.90,3260.20,15.0,V40N,ATH,3850.0,48903.00,49950.00,50248.50,57750.0,-0.59,2.14,18.09,15.95,1.48,3.0
4,1643.75,BAJAJFINSV.NS,1905.0,1610.25,1454.93,52.0,V40,ATH,1905.0,75656.36,85475.00,83733.00,99060.0,2.08,12.98,30.93,17.95,2.29,30.0
21,1366.30,ICICIGI.NS,1613.0,1365.20,1177.58,84.0,V40,ATH,1613.0,98916.72,114769.20,114676.80,135492.0,0.08,16.03,36.98,20.95,3.00,48.0
20,1686.10,HDFCBANK.NS,1728.0,1678.90,1683.55,30.0,V40,NaN,2050.0,50506.50,50583.00,50367.00,61500.0,0.43,0.15,21.77,21.62,1.53,0.0
26,32.93,ITBEES.NS,40.0,32.58,31.49,3350.0,AR,ATH,40.0,105491.50,110315.50,109143.00,134000.0,1.07,4.57,27.02,22.45,3.20,15.0
24,1473.10,INFY.NS,1848.0,1422.95,1319.64,81.0,V40,ATH,1848.0,106890.84,119321.10,115258.95,149688.0,3.52,11.63,40.04,28.41,3.24,38.0
8,4670.65,BAYERCROP.NS,6002.0,4689.15,4346.57,16.0,V40N,ATH,6002.0,69545.12,74730.40,75026.40,96032.0,-0.39,7.46,38.09,30.63,2.11,16.0
7,1700.05,BATAINDIA.NS,2173.0,1692.95,1523.62,100.0,V40,ATH,2173.0,152362.00,170005.00,169295.00,217300.0,0.42,11.58,42.62,31.04,4.62,53.0


In [11]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Target,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
18,1187.60,GLAND.NS,4316.0,1171.40,1225.87,55.0,VR,2T3Y,4316.0,67422.85,65318.00,64427.00,237380.0,1.38,-3.12,252.08,255.20,2.04,-6.0
47,508.35,VALIANTORG.NS,1851.0,500.35,532.61,185.0,VR,2T3Y,1851.0,98532.85,94044.75,92564.75,342435.0,1.60,-4.55,247.53,252.08,2.99,-14.0
30,1484.45,LUXIND.NS,4546.0,1498.70,1493.72,51.0,V40N,2T3Y,4546.0,76179.72,75706.95,76433.70,231846.0,-0.95,-0.62,204.34,204.96,2.31,-1.0
46,343.80,VAIBHAVGBL.NS,966.0,330.00,317.48,144.0,VR,2T3Y,966.0,45717.12,49507.20,47520.00,139104.0,4.18,8.29,204.27,195.98,1.39,12.0
6,214.50,BANDHANBNK.NS,723.0,215.35,244.52,400.0,VR,2T3Y,723.0,97808.00,85800.00,86140.00,289200.0,-0.39,-12.28,195.68,207.96,2.96,-36.0
31,1439.00,METROPOLIS.NS,3440.0,1462.65,1365.98,22.0,VR,ATH,3440.0,30051.56,31658.00,32178.30,75680.0,-1.62,5.35,151.83,146.48,0.91,5.0
32,116.25,NIACL.NS,287.0,116.90,117.07,390.0,VR,ATH,287.0,45657.30,45337.50,45591.00,111930.0,-0.56,-0.70,145.15,145.85,1.38,-1.0
44,2466.60,TEAMLEASE.NS,5281.0,2451.55,2279.93,21.0,V40N,2T3Y,5281.0,47878.53,51798.60,51482.55,110901.0,0.61,8.19,131.63,123.44,1.45,12.0
22,1489.75,INDIGOPNTS.NS,3112.0,1485.50,1407.73,124.0,V40N,2T3Y,3112.0,174558.52,184729.00,184202.00,385888.0,0.29,5.83,121.07,115.24,5.29,31.0
1,1546.60,AAVAS.NS,3196.0,1531.05,1453.92,37.0,VR,2T3Y,3196.0,53795.04,57224.20,56648.85,118252.0,1.02,6.37,119.82,113.45,1.63,10.0


In [12]:
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Target,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
22,1489.75,INDIGOPNTS.NS,3112.0,1485.50,1407.73,124.0,V40N,2T3Y,3112.0,174558.52,184729.00,184202.00,385888.0,0.29,5.83,121.07,115.24,5.29,31.0
34,36465.00,PAGEIND.NS,53549.0,36626.05,38548.77,4.0,V40,ATH,53549.0,154195.08,145860.00,146504.20,214196.0,-0.44,-5.41,38.91,44.32,4.67,-25.0
7,1700.05,BATAINDIA.NS,2173.0,1692.95,1523.62,100.0,V40,ATH,2173.0,152362.00,170005.00,169295.00,217300.0,0.42,11.58,42.62,31.04,4.62,53.0
24,1473.10,INFY.NS,1848.0,1422.95,1319.64,81.0,V40,ATH,1848.0,106890.84,119321.10,115258.95,149688.0,3.52,11.63,40.04,28.41,3.24,38.0
26,32.93,ITBEES.NS,40.0,32.58,31.49,3350.0,AR,ATH,40.0,105491.50,110315.50,109143.00,134000.0,1.07,4.57,27.02,22.45,3.20,15.0
21,1366.30,ICICIGI.NS,1613.0,1365.20,1177.58,84.0,V40,ATH,1613.0,98916.72,114769.20,114676.80,135492.0,0.08,16.03,36.98,20.95,3.00,48.0
47,508.35,VALIANTORG.NS,1851.0,500.35,532.61,185.0,VR,2T3Y,1851.0,98532.85,94044.75,92564.75,342435.0,1.60,-4.55,247.53,252.08,2.99,-14.0
19,2414.80,HDFCAMC.NS,3518.0,2383.70,1973.85,50.0,V40,ATH,3518.0,98692.50,120740.00,119185.00,175900.0,1.30,22.34,78.23,55.89,2.99,67.0
6,214.50,BANDHANBNK.NS,723.0,215.35,244.52,400.0,VR,2T3Y,723.0,97808.00,85800.00,86140.00,289200.0,-0.39,-12.28,195.68,207.96,2.96,-36.0
17,5730.00,GILLETTE.NS,7529.0,5873.25,4475.85,21.0,V40,ATH,7529.0,93992.85,120330.00,123338.25,158109.0,-2.44,28.02,68.21,40.19,2.85,80.0


In [13]:
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  tmp_df.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [14]:
tmp_df.sort_values(by = 'Risk Ind', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Target,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
6,214.50,BANDHANBNK.NS,723.0,215.35,244.52,400.0,VR,2T3Y,723.0,97808.00,85800.00,86140.00,289200.0,-0.39,-12.28,195.68,207.96,2.96,-36.0
34,36465.00,PAGEIND.NS,53549.0,36626.05,38548.77,4.0,V40,ATH,53549.0,154195.08,145860.00,146504.20,214196.0,-0.44,-5.41,38.91,44.32,4.67,-25.0
37,517.20,RAJESHEXPO.NS,949.0,520.15,578.92,87.0,V40N,ATH,949.0,50366.04,44996.40,45253.05,82563.0,-0.57,-10.66,63.93,74.59,1.53,-16.0
47,508.35,VALIANTORG.NS,1851.0,500.35,532.61,185.0,VR,2T3Y,1851.0,98532.85,94044.75,92564.75,342435.0,1.60,-4.55,247.53,252.08,2.99,-14.0
33,112.00,NMDC.NS,178.0,113.50,122.55,265.0,VR,ATH,178.0,32475.75,29680.00,30077.50,47170.0,-1.32,-8.61,45.25,53.86,0.98,-8.0
18,1187.60,GLAND.NS,4316.0,1171.40,1225.87,55.0,VR,2T3Y,4316.0,67422.85,65318.00,64427.00,237380.0,1.38,-3.12,252.08,255.20,2.04,-6.0
11,1317.10,CLEAN.NS,2648.0,1318.00,1365.11,39.0,VR,2T3Y,2648.0,53239.29,51366.90,51402.00,103272.0,-0.07,-3.52,93.98,97.50,1.61,-6.0
3,419.00,AMBUJACEM.NS,581.0,416.60,424.02,180.0,VR,ATH,581.0,76323.60,75420.00,74988.00,104580.0,0.58,-1.18,37.02,38.20,2.31,-3.0
25,625.40,IRCTC.NS,1164.0,625.55,633.05,120.0,AR,ATH,1164.0,75966.00,75048.00,75066.00,139680.0,-0.02,-1.21,83.87,85.08,2.30,-3.0
9,2279.30,CAMS.NS,3867.0,2272.20,2324.85,13.0,VR,ATH,3867.0,30223.05,29630.90,29538.60,50271.0,0.31,-1.96,66.33,68.29,0.92,-2.0


In [15]:
print('Total profit: ',round(sum(tmp_df[tmp_df['Current P/L%'] > 0]['Current']) - sum(tmp_df[tmp_df['Current P/L%'] > 0]['Investment']),0))
print('Total loss: ',round(sum(tmp_df[tmp_df['Current P/L%'] < 0]['Current']) - sum(tmp_df[tmp_df['Current P/L%'] < 0]['Investment']),0))

Total profit:  223774.0
Total loss:  -41182.0


In [16]:
# SIP based on portfolio
tmp_df[tmp_df['Weightage%'] < 2].sort_values(by = 'Current P/L%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Target,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
37,517.20,RAJESHEXPO.NS,949.0,520.15,578.92,87.0,V40N,ATH,949.0,50366.04,44996.40,45253.05,82563.0,-0.57,-10.66,63.93,74.59,1.53,-16.0
33,112.00,NMDC.NS,178.0,113.50,122.55,265.0,VR,ATH,178.0,32475.75,29680.00,30077.50,47170.0,-1.32,-8.61,45.25,53.86,0.98,-8.0
11,1317.10,CLEAN.NS,2648.0,1318.00,1365.11,39.0,VR,2T3Y,2648.0,53239.29,51366.90,51402.00,103272.0,-0.07,-3.52,93.98,97.50,1.61,-6.0
14,414.00,EMAMILTD.NS,588.0,418.40,422.59,72.0,V200,ATH,588.0,30426.48,29808.00,30124.80,42336.0,-1.05,-2.03,39.14,41.17,0.92,-2.0
9,2279.30,CAMS.NS,3867.0,2272.20,2324.85,13.0,VR,ATH,3867.0,30223.05,29630.90,29538.60,50271.0,0.31,-1.96,66.33,68.29,0.92,-2.0
5,16.95,BAJAJHIND.NS,24.0,17.25,17.15,1750.0,VR,ATH,24.0,30012.50,29662.50,30187.50,42000.0,-1.74,-1.17,39.94,41.11,0.91,-1.0
32,116.25,NIACL.NS,287.0,116.90,117.07,390.0,VR,ATH,287.0,45657.30,45337.50,45591.00,111930.0,-0.56,-0.70,145.15,145.85,1.38,-1.0
40,90.80,SAIL.NS,130.0,91.60,90.90,330.0,VR,ATH,130.0,29997.00,29964.00,30228.00,42900.0,-0.87,-0.11,43.01,43.12,0.91,-0.0
20,1686.10,HDFCBANK.NS,1728.0,1678.90,1683.55,30.0,V40,NaN,2050.0,50506.50,50583.00,50367.00,61500.0,0.43,0.15,21.77,21.62,1.53,0.0
29,4999.95,LTIM.NS,7380.0,5134.85,4987.59,10.0,V200,ATH,7380.0,49875.90,49999.50,51348.50,73800.0,-2.63,0.25,47.97,47.72,1.51,0.0


In [17]:
myts[myts['Symbol'] == 'INDIGOPNTS']

,Scrip Name,Symbol,Date,Type,Qty,Price,Value
19,Indigo Paints,INDIGOPNTS,26-Apr-23,Buy,22,"1,148.85","25,274.70"
24,Indigo Paints,INDIGOPNTS,02-May-23,Buy,22,"1,155.75","25,426.50"
42,Indigo Paints,INDIGOPNTS,09-May-23,Buy,39,"1,287.63","50,217.57"
66,Indigo Paints,INDIGOPNTS,22-May-23,Buy,83,"1,408.25","1,16,884.75"
82,Indigo Paints,INDIGOPNTS,31-May-23,Sell,83,"1,457.19","1,20,946.77"
106,Indigo Paints,INDIGOPNTS,12-Jun-23,Buy,41,"1,406.69","57,674.29"


In [18]:
myts[(myts['Symbol'] == 'INDIGOPNTS') & (myts['Type'] == 'Buy')]

,Scrip Name,Symbol,Date,Type,Qty,Price,Value
19,Indigo Paints,INDIGOPNTS,26-Apr-23,Buy,22,"1,148.85","25,274.70"
24,Indigo Paints,INDIGOPNTS,02-May-23,Buy,22,"1,155.75","25,426.50"
42,Indigo Paints,INDIGOPNTS,09-May-23,Buy,39,"1,287.63","50,217.57"
66,Indigo Paints,INDIGOPNTS,22-May-23,Buy,83,"1,408.25","1,16,884.75"
106,Indigo Paints,INDIGOPNTS,12-Jun-23,Buy,41,"1,406.69","57,674.29"


In [19]:
myts.groupby(['Symbol','Type'])['Qty'].aggregate(np.sum).reset_index()

,Symbol,Type,Qty
0,5PAISA,Buy,228
1,AAVAS,Buy,23
2,AMARAJABAT,Buy,72
3,AMBUJACEM,Buy,180
4,ASIANPAINT,Buy,41
...,...,...,...
62,VAIBHAVGBL,Buy,204
63,VAIBHAVGBL,Sell,60
64,VALIANTORG,Buy,184
65,VALIANTORG,Sell,40
